# Purpose
### In this analysis, I will try to find out what kind of drugs are used together by conducting association analysis. Data will use the entire data for 2019.

# Import Modules

In [1]:
import json  
import pandas as pd  
from pandas.io.json import json_normalize  
import numpy as np
import time

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Data Load

In [2]:

def loading_data(temp):
    data = pd.DataFrame()
    data['safetyreportid'] = temp['safetyreportid']  # read 'safetyreportid' column
        
    patientdrug = temp['patient.drug'].apply(lambda x: x[0])  # read 'patientdrug' column
    data['openfda'] = patientdrug.apply(lambda x: x['openfda'] if 'openfda' in x.keys() else np.nan) # read 'openfda' line from dictionary
    data['generic_name'] = data['openfda'].apply(lambda x: x if type(x) != dict else (x['generic_name'] if 'generic_name' in x.keys() else np.nan)) # read 'generic_name' line from 'openfda' 
     
    stack = data.apply(lambda x: pd.Series(x['generic_name']), axis=1).stack().reset_index(level=1, drop=True) # split generic_name(list) to each different row
    stack.name = 'generic_name' # set the name
    data = data.drop('generic_name', axis=1) # drop generic_name column
    data = data.join(stack)
        
    data.drop('openfda', axis = 1, inplace=True)  # remove unnecessary columns
    return data

In [3]:
start = time.time() 

df = pd.DataFrame()
for i in range(1, 134):
    a = 'data (' + str(i) + ').json'
    with open(a) as f:
        d = json.load(f) 
    result = json_normalize(d['results']) 
    temp = loading_data(result)
    df = pd.concat([df, temp], sort=False)

    
end = time.time()
(end - start)/60

96.9050016840299

In [4]:
# df.to_csv('C:/AstraZeneca/market.csv')

In [4]:
# df = pd.read_csv('C:/AstraZeneca/market.csv', index_col=0)

# Data Cleansing

### Missing Values

In [5]:
# Check missing value
df.isna().sum() / len(df)

safetyreportid    0.000000
generic_name      0.338379
dtype: float64

In [6]:
# Drop Missing Values
df.dropna(inplace = True)

In [7]:
# Count number of generic name by an ID

id_cnt = df['safetyreportid'].value_counts()
df.set_index('safetyreportid', inplace = True)

# Bring data having two or more generic names because data having only one generic name is meaningless in association
df = df[id_cnt > 1].reset_index()

C:\Users\Sihyun Kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


### String Manipulation

In [8]:
# Some data have 'comma' and 'AND' in  a generic_name column. Replace 'AND' to 'comma' to make it List.

df['generic_name'] = df['generic_name'].apply(lambda x: x.replace(", AND", ","))   
df['generic_name'] = df['generic_name'].apply(lambda x: x.replace(" AND", ","))   

In [9]:
# # Change string to List and split each element into different rows
# df = pd.DataFrame(df['generic_name'].str.split(',').tolist(), index = df['safetyreportid']).stack().reset_index().drop('level_1', axis=1)

# # Rename columns
# df.columns = ['safetyreportid', 'generic_name']


# Remove space to make it clear (Some data have space while others don't have it in spite of the same generic name)

df['generic_name'] = df['generic_name'].apply(lambda x: x.replace(" ", ""))   

# Pivot Table for Association Analysis

In [10]:
# Add vaue columns for aggregation

df['values'] = 1

In [11]:
# Group by to make it smaller

df = df.groupby(['safetyreportid','generic_name'])['values'].sum().reset_index()

In [12]:
df

,safetyreportid,generic_name,values
0,10004876,ATORVASTATINCALCIUM,1
1,10004876,"ATORVASTATINCALCIUM,FILMCOATED",1
2,10011709,ATORVASTATINCALCIUM,1
3,10011709,"ATORVASTATINCALCIUM,FILMCOATED",1
4,10012294,ATORVASTATINCALCIUM,1
...,...,...,...
274151,9992206,"ATORVASTATINCALCIUM,FILMCOATED",1
274152,9992545,LITHIUM,1
274153,9992545,LITHIUMBROMATUM,1
274154,9996139,ATORVASTATINCALCIUM,1


In [13]:
# Pivot Table

table = pd.pivot_table(df, values='values', index=['safetyreportid'],
                    columns=['generic_name'], aggfunc=np.sum).fillna(0)

In [14]:
# Encode_units function to have only 0 or 1

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1    

In [15]:
table = table.applymap(encode_units)

In [16]:
# Final table
table.head(20)

generic_name,(METHYLPHENIDATEHYDROCHLORIDE),(SALINE),0XYGEN,383IBUPROFEN200MG,ABACAVIR,"ABACAVIR,LAMIVUDINE",ABACAVIRSULFATE,"ABACAVIRSULFATE,LAMIVUDINE",ABIRATERONE,ABIRATERONEACETATE,...,ZICONOTIDE,ZICONOTIDEACETATE,ZINC,"ZINCACETATEANHYDROUS,ZINCGLUCONATE",ZINCGLUCONATE,ZINCOXIDE,ZINCOXIDESUNSCREEN,ZINCOXIDESUNSREEN,ZIPRASIDONEHCL,ZIPRASIDONEHYDROCHLORIDE
safetyreportid,,,,,,,,,,,,,,,,,,,,,
10004876,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10011709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10012294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10015666,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10017132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10018841,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10021571,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10021826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10022238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Association Analysis

In [17]:
frequent_items = apriori(table, min_support=0.01, use_colnames=True)

In [18]:
# Count number of itemsets

frequent_items_list = frequent_items
frequent_items_list['itemsets'] = frequent_items_list['itemsets'].apply(lambda x: list(x))
frequent_items_list['cnt'] = frequent_items_list['itemsets'].apply(lambda x: len(x))

In [21]:
# Check the result having two or more itemsets sorted by a value of support

frequent_items_list[frequent_items_list['cnt'] > 1].sort_values('support', ascending = False).head(30)

,support,itemsets,cnt
111,0.075765,"[METHOTREXATE, METHOTREXATESODIUM]",2
92,0.061329,"[ESOMEPRAZOLESODIUM, ESOMEPRAZOLEMAGNESIUM]",2
96,0.041010,"[IBUPROFEN, IBUPROFEN200MG]",2
95,0.041010,"[IBUPROFEN200MG, IBUPFROFEN]",2
94,0.041010,"[IBUPROFEN, IBUPFROFEN]",2
129,0.041010,"[IBUPROFEN, IBUPROFEN200MG, IBUPFROFEN]",3
83,0.039206,"[DEXAMETHASONE, DEXAMETHASONE1.5MG]",2
88,0.036516,"[DOCETAXELANHYDROUS, DOCETAXEL]",2
100,0.033879,"[LEVETIRACETAMINJECTION, LEVETIRACETAM]",2
87,0.031987,"[IBUPROFEN200MG, DIPHENHYDRAMINECITRATE,IBUPRO...",2


In [20]:
# Association Rules to check Lift and Confidence

rules = association_rules(frequent_items, metric="lift", min_threshold=1)
rules.sort_values('lift', ascending = False).head(30)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
91,(NAPROXENSODIUM),(NAPROXENSODIUM220MG),0.010081,0.010081,0.010081,1.0,99.194492,0.009980,inf
90,(NAPROXENSODIUM220MG),(NAPROXENSODIUM),0.010081,0.010081,0.010081,1.0,99.194492,0.009980,inf
214,"(METFORMINHYDROCHLORIDETABLETS, METFORMINHYDRO...",(METFORMINHYDROCHLORIDETABLET),0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
222,(METFORMINHYDROCHLORIDETABLETS),"(METFORMINHYDROCHLORIDETABLET, METFORMINHYDROC...",0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
221,(METFORMINHYDROCHLORIDETABLET),"(METFORMINHYDROCHLORIDETABLETS, METFORMINHYDRO...",0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
220,"(METFORMINHYDROCHLORIDETABLETS, METFORMINHYDRO...",(METFORMINHYDROCHLORIDETABLET),0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
219,"(METFORMINHYDROCHLORIDETABLET, METFORMINHYDROC...",(METFORMINHYDROCHLORIDETABLETS),0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
218,"(METFORMINHYDROCHLORIDETABLET, METFORMINHYDROC...",(METFORMINHYDROCHLORIDEEXTENDED-RELEASETABLETS),0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
217,(METFORMINHYDROCHLORIDE),"(METFORMINHYDROCHLORIDETABLET, METFORMINHYDROC...",0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
216,(METFORMINHYDROCHLORIDETABLETS),"(METFORMINHYDROCHLORIDETABLET, METFORMINHYDROC...",0.010359,0.010359,0.010359,1.0,96.536013,0.010252,inf
